In [1]:
# # Helper functions for the parallel computing 
import pandas as pd
import numpy as np

import random
import os
import sys
import psutil

import matplotlib
import matplotlib.pyplot as plt
import math
from multiprocessing import cpu_count,Pool 
import multiprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
import pygeohash as gh
import pickle
from datetime import date

import pysolar
from pysolar.solar import *
from datetime import date
import datetime as dt
import pygeohash as gh
from pvlib import atmosphere
import pvlib.solarposition as pv
from pvlib.tools import datetime_to_djd, djd_to_datetime
# X=pd.read_csv('/home/optim7/Dropbox/Smart_Analytics_work/WeatherData_WOOPA/PV_data/Testing123.csv',index_col='Datetime')
# X.index = pd.to_datetime(X.index,unit='s')
# X.head()
import pytz
import calendar
import numpy as np



class WithExtraArgs(object):
     def __init__(self, func, **args):
         self.func = func
         self.args = args
     def __call__(self, df):
         return self.func(df, **self.args)

def applyParallel(data, func,pool,partition,kwargs):
     data_split = [data[i:i + partition] for i in range(0, len(data), partition)]
     #data_split = np.array_split(data, min(partitions,data.shape[0]))
     data =pool.map(WithExtraArgs(func, **kwargs), data_split)
     #data = pd.concat(pool.map(WithExtraArgs(func, **kwargs), data_split))
     return data

# merge acc and non acc

In [2]:
city='new_method/hann/clustering/gridgrowing6'
print(city)

new_method/hann/clustering/gridgrowing6


In [3]:
hann_test_nonacc=pd.read_csv("/data/dadwal/data/DAP_data/dataPrepTrainTestCluster/Baveria/"+city+"/test_nonacc.csv",header=0)
hann_test_nonacc['count']=0
hann_test_nonacc.columns=['geohash','lat','long','UJAHR','UMONAT','UWOCHENTAG','hour','geohash6','cluster_id','counts']
hann_test_nonacc=hann_test_nonacc[['geohash6','lat','long','UJAHR','UMONAT','UWOCHENTAG','hour','cluster_id','counts']]
hann_test_nonacc.columns=['geohash','lat','long','UJAHR','UMONAT','UWOCHENTAG','hour','cluster_id','counts']
hann_test_nonacc

,geohash,lat,long,UJAHR,UMONAT,UWOCHENTAG,hour,cluster_id,counts
0,u1r400,52.386882,9.849992,2019,6,7,8,2272.0,0
1,u1r18n,52.333173,9.844283,2019,10,6,8,869.0,0
2,u1qfh3,52.393468,9.680263,2019,10,3,20,1364.0,0
3,u1qf5r,52.425890,9.642203,2019,12,7,3,1273.0,0
4,u1r1bj,52.367538,9.845551,2019,8,3,8,-1.0,0
...,...,...,...,...,...,...,...,...,...
4435,u1r40s,52.405308,9.868922,2019,11,3,6,2302.0,0
4436,u1qfhz,52.424450,9.706125,2019,11,1,5,1499.0,0
4437,u1qcv7,52.355909,9.724261,2019,7,1,14,165.0,0
4438,u1qfjk,52.409719,9.724982,2019,8,5,3,1740.0,0


In [5]:
city2='new_method/hann'
print(city2)

new_method/hann


In [6]:
hann_test_acc=pd.read_csv('/data/dadwal/data/DAP_data/dataPrepTrainTestCluster/Baveria/'+city+'/test_acc.csv',header=0)
hann_test_acc

,acc_id,acc_lat,acc_long,UJAHR,UMONAT,USTUNDE,UWOCHENTAG,STRZUSTAND,UART,UTYP1,geohash,sessiondivided,cluster_id
0,79231_2019,52.414667,9.641973,2019,6,14,4,0,2,6,u1qf5m,14-16,1234.0
1,79232_2019,52.368081,9.742417,2019,6,16,4,0,3,6,u1qcvt,16-18,165.0
2,79233_2019,52.376961,9.661384,2019,6,17,4,0,5,3,u1qcgy,16-18,11.0
3,79234_2019,52.374535,9.694944,2019,6,13,4,0,0,7,u1qcuw,12-14,269.0
4,79236_2019,52.369425,9.861861,2019,6,17,4,0,2,6,u1r1bm,16-18,2244.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1762,229027_2019,52.399756,9.780113,2019,12,9,5,1,5,3,u1qfne,8-10,1849.0
1763,229030_2019,52.451460,9.743495,2019,12,13,5,0,5,3,u1qfms,12-14,-1.0
1764,229031_2019,52.332190,9.809078,2019,12,18,5,0,0,7,u1qcxn,18-20,838.0
1765,229032_2019,52.337201,9.760379,2019,12,14,5,0,5,3,u1qcwp,14-16,745.0


In [7]:
city1='new_method/hann'
data_train_accActual=pd.read_csv("/data/dadwal/data/DAP_data/dataPrepTrainTestCluster/Baveria/"+city1+"/test_acc_actual.csv",header=0)
data_train_accActual=data_train_accActual[['acc_id']]
data_train_accActual

,acc_id
0,79231_2019
1,79232_2019
2,79233_2019
3,79234_2019
4,79236_2019
...,...
1475,229026_2019
1476,229027_2019
1477,229031_2019
1478,229032_2019


In [8]:
hann_true_acc=pd.merge(hann_test_acc,data_train_accActual,on='acc_id',)
hann_true_acc

,acc_id,acc_lat,acc_long,UJAHR,UMONAT,USTUNDE,UWOCHENTAG,STRZUSTAND,UART,UTYP1,geohash,sessiondivided,cluster_id
0,79231_2019,52.414667,9.641973,2019,6,14,4,0,2,6,u1qf5m,14-16,1234.0
1,79232_2019,52.368081,9.742417,2019,6,16,4,0,3,6,u1qcvt,16-18,165.0
2,79233_2019,52.376961,9.661384,2019,6,17,4,0,5,3,u1qcgy,16-18,11.0
3,79234_2019,52.374535,9.694944,2019,6,13,4,0,0,7,u1qcuw,12-14,269.0
4,79236_2019,52.369425,9.861861,2019,6,17,4,0,2,6,u1r1bm,16-18,2244.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1475,229026_2019,52.406406,9.731526,2019,12,17,5,0,1,6,u1qfjk,16-18,1737.0
1476,229027_2019,52.399756,9.780113,2019,12,9,5,1,5,3,u1qfne,8-10,1849.0
1477,229031_2019,52.332190,9.809078,2019,12,18,5,0,0,7,u1qcxn,18-20,838.0
1478,229032_2019,52.337201,9.760379,2019,12,14,5,0,5,3,u1qcwp,14-16,745.0


In [9]:
hann_true_acc['count']=1
hann_true_acc=hann_true_acc[['geohash','acc_lat','acc_long','UJAHR','UMONAT','UWOCHENTAG','USTUNDE','cluster_id','count']]
hann_true_acc.columns=['geohash','lat','long','UJAHR','UMONAT','UWOCHENTAG','hour','cluster_id','counts']
hann_true_acc

,geohash,lat,long,UJAHR,UMONAT,UWOCHENTAG,hour,cluster_id,counts
0,u1qf5m,52.414667,9.641973,2019,6,4,14,1234.0,1
1,u1qcvt,52.368081,9.742417,2019,6,4,16,165.0,1
2,u1qcgy,52.376961,9.661384,2019,6,4,17,11.0,1
3,u1qcuw,52.374535,9.694944,2019,6,4,13,269.0,1
4,u1r1bm,52.369425,9.861861,2019,6,4,17,2244.0,1
...,...,...,...,...,...,...,...,...,...
1475,u1qfjk,52.406406,9.731526,2019,12,5,17,1737.0,1
1476,u1qfne,52.399756,9.780113,2019,12,5,9,1849.0,1
1477,u1qcxn,52.332190,9.809078,2019,12,5,18,838.0,1
1478,u1qcwp,52.337201,9.760379,2019,12,5,14,745.0,1


In [10]:
frame=[hann_true_acc,hann_test_nonacc]
test_generted=pd.concat(frame)
test_generted

,geohash,lat,long,UJAHR,UMONAT,UWOCHENTAG,hour,cluster_id,counts
0,u1qf5m,52.414667,9.641973,2019,6,4,14,1234.0,1
1,u1qcvt,52.368081,9.742417,2019,6,4,16,165.0,1
2,u1qcgy,52.376961,9.661384,2019,6,4,17,11.0,1
3,u1qcuw,52.374535,9.694944,2019,6,4,13,269.0,1
4,u1r1bm,52.369425,9.861861,2019,6,4,17,2244.0,1
...,...,...,...,...,...,...,...,...,...
4435,u1r40s,52.405308,9.868922,2019,11,3,6,2302.0,0
4436,u1qfhz,52.424450,9.706125,2019,11,1,5,1499.0,0
4437,u1qcv7,52.355909,9.724261,2019,7,1,14,165.0,0
4438,u1qfjk,52.409719,9.724982,2019,8,5,3,1740.0,0


In [11]:
def geohash(lat,long):
    geo=gh.encode(float(lat), float(long), precision=7)
    return geo
test_generted['geohash7'] = test_generted.apply(lambda row: geohash(row['lat'],row['long']), axis=1)
test_generted

,geohash,lat,long,UJAHR,UMONAT,UWOCHENTAG,hour,cluster_id,counts,geohash7
0,u1qf5m,52.414667,9.641973,2019,6,4,14,1234.0,1,u1qf5mv
1,u1qcvt,52.368081,9.742417,2019,6,4,16,165.0,1,u1qcvtq
2,u1qcgy,52.376961,9.661384,2019,6,4,17,11.0,1,u1qcgyg
3,u1qcuw,52.374535,9.694944,2019,6,4,13,269.0,1,u1qcuw7
4,u1r1bm,52.369425,9.861861,2019,6,4,17,2244.0,1,u1r1bmt
...,...,...,...,...,...,...,...,...,...,...
4435,u1r40s,52.405308,9.868922,2019,11,3,6,2302.0,0,u1r40s4
4436,u1qfhz,52.424450,9.706125,2019,11,1,5,1499.0,0,u1qfhze
4437,u1qcv7,52.355909,9.724261,2019,7,1,14,165.0,0,u1qcv70
4438,u1qfjk,52.409719,9.724982,2019,8,5,3,1740.0,0,u1qfjkc


In [12]:
test_minus1=test_generted.loc[test_generted['cluster_id']==-1]
test_minus1

,geohash,lat,long,UJAHR,UMONAT,UWOCHENTAG,hour,cluster_id,counts,geohash7
82,u1r1be,52.356979,9.871445,2019,6,3,16,-1.0,1,u1r1bek
676,u1qcv8,52.344134,9.739338,2019,6,1,9,-1.0,1,u1qcv8g
779,u1qfhp,52.421685,9.677881,2019,7,2,5,-1.0,1,u1qfhpp
971,u1qfqe,52.443834,9.787148,2019,8,4,10,-1.0,1,u1qfqen
1319,u1qfjg,52.402592,9.754312,2019,11,7,7,-1.0,1,u1qfjgw
...,...,...,...,...,...,...,...,...,...,...
4398,u1r1bz,52.380574,9.877202,2019,9,7,9,-1.0,0,u1r1bz8
4410,u1qfqs,52.448956,9.785237,2019,7,6,4,-1.0,0,u1qfqsj
4418,u1r1cp,52.378341,9.888617,2019,8,7,15,-1.0,0,u1r1cp0
4432,u1qcxj,52.323533,9.802804,2019,8,5,18,-1.0,0,u1qcxj4


# here assigining -1 cluster to geohash id

In [13]:
#city='new_method/Nurmberg/clustering/gridgrowing6'
#print(city)
print(city)

new_method/hann/clustering/gridgrowing6


In [14]:
geohash_ind=pd.read_csv("/data/dadwal/data/DAP_data/dataPrepTrainTestCluster/Baveria/"+city+"/minus1geohash.csv")
geohash_ind

,geohash,cluster_id
0,u1qcg6,2342.0
1,u1qcg7,2343.0
2,u1qcgd,2344.0
3,u1qcge,2345.0
4,u1qcgg,2346.0
...,...,...
274,u1r41k,2616.0
275,u1r41m,2617.0
276,u1r41n,2618.0
277,u1r41p,2619.0


In [15]:
meger_minus1_nonacc=pd.merge(test_minus1,geohash_ind,on='geohash',how='left')
meger_minus1_nonacc.drop(['cluster_id_x'],axis=1,inplace=True)
meger_minus1_nonacc=meger_minus1_nonacc[['geohash', 'lat', 'long', 'UJAHR', 'UMONAT', 'UWOCHENTAG',
        'hour', 'cluster_id_y','counts' ]]
meger_minus1_nonacc.columns=['geohash', 'lat', 'long', 'UJAHR', 'UMONAT', 'UWOCHENTAG', 'hour', 'cluster_id','counts' ]
meger_minus1_nonacc

,geohash,lat,long,UJAHR,UMONAT,UWOCHENTAG,hour,cluster_id,counts
0,u1r1be,52.356979,9.871445,2019,6,3,16,2563.0,1
1,u1qcv8,52.344134,9.739338,2019,6,1,9,2375.0,1
2,u1qfhp,52.421685,9.677881,2019,7,2,5,2466.0,1
3,u1qfqe,52.443834,9.787148,2019,8,4,10,2519.0,1
4,u1qfjg,52.402592,9.754312,2019,11,7,7,2470.0,1
...,...,...,...,...,...,...,...,...,...
955,u1r1bz,52.380574,9.877202,2019,9,7,9,2578.0,0
956,u1qfqs,52.448956,9.785237,2019,7,6,4,2523.0,0
957,u1r1cp,52.378341,9.888617,2019,8,7,15,2582.0,0
958,u1qcxj,52.323533,9.802804,2019,8,5,18,2394.0,0


In [16]:
test_generted_others=test_generted.loc[test_generted['cluster_id']>-1]
test_generted_others=test_generted_others[['geohash7', 'lat', 'long', 'UJAHR', 'UMONAT', 'UWOCHENTAG',
        'hour', 'cluster_id','counts' ]]
test_generted_others.columns=['geohash', 'lat', 'long', 'UJAHR', 'UMONAT', 'UWOCHENTAG', 'hour', 'cluster_id','counts' ]
test_generted_others

,geohash,lat,long,UJAHR,UMONAT,UWOCHENTAG,hour,cluster_id,counts
0,u1qf5mv,52.414667,9.641973,2019,6,4,14,1234.0,1
1,u1qcvtq,52.368081,9.742417,2019,6,4,16,165.0,1
2,u1qcgyg,52.376961,9.661384,2019,6,4,17,11.0,1
3,u1qcuw7,52.374535,9.694944,2019,6,4,13,269.0,1
4,u1r1bmt,52.369425,9.861861,2019,6,4,17,2244.0,1
...,...,...,...,...,...,...,...,...,...
4434,u1qfkb9,52.429673,9.703170,2019,11,2,5,1806.0,0
4435,u1r40s4,52.405308,9.868922,2019,11,3,6,2302.0,0
4436,u1qfhze,52.424450,9.706125,2019,11,1,5,1499.0,0
4437,u1qcv70,52.355909,9.724261,2019,7,1,14,165.0,0


In [17]:
frames=[meger_minus1_nonacc,test_generted_others]
test_generted=pd.concat(frames)
test_generted=test_generted.reset_index(drop=True)
test_generted

,geohash,lat,long,UJAHR,UMONAT,UWOCHENTAG,hour,cluster_id,counts
0,u1r1be,52.356979,9.871445,2019,6,3,16,2563.0,1
1,u1qcv8,52.344134,9.739338,2019,6,1,9,2375.0,1
2,u1qfhp,52.421685,9.677881,2019,7,2,5,2466.0,1
3,u1qfqe,52.443834,9.787148,2019,8,4,10,2519.0,1
4,u1qfjg,52.402592,9.754312,2019,11,7,7,2470.0,1
...,...,...,...,...,...,...,...,...,...
5915,u1qfkb9,52.429673,9.703170,2019,11,2,5,1806.0,0
5916,u1r40s4,52.405308,9.868922,2019,11,3,6,2302.0,0
5917,u1qfhze,52.424450,9.706125,2019,11,1,5,1499.0,0
5918,u1qcv70,52.355909,9.724261,2019,7,1,14,165.0,0


# geohash assigining ends here

In [19]:
print(city)

new_method/hann/clustering/gridgrowing6


In [20]:
latlong=pd.read_csv('/data/dadwal/data/DAP_data/dataPrepTrainTestCluster/Baveria/'+city+'/grid6_centroid_cluster.csv',header=0)
latlong

,cluster_id,middlelat,middlelong
0,0.0,52.351302,9.655861
1,2.0,52.358348,9.652560
2,3.0,52.352814,9.660477
3,5.0,52.356783,9.659970
4,9.0,52.355317,9.666380
...,...,...,...
728,2616.0,52.407273,9.905866
729,2617.0,52.412861,9.901129
730,2618.0,52.419861,9.888476
731,2619.0,52.422158,9.888607


In [23]:
test_non_acc=pd.merge(test_generted,latlong,on='cluster_id',how='left')
test_non_acc=test_non_acc.fillna(0)
test_non_acc

,geohash,lat,long,UJAHR,UMONAT,UWOCHENTAG,hour,cluster_id,counts,middlelat,middlelong
0,u1r1be,52.356979,9.871445,2019,6,3,16,2563.0,1,52.357380,9.873714
1,u1qcv8,52.344134,9.739338,2019,6,1,9,2375.0,1,52.341971,9.739637
2,u1qfhp,52.421685,9.677881,2019,7,2,5,2466.0,1,52.424523,9.676540
3,u1qfqe,52.443834,9.787148,2019,8,4,10,2519.0,1,52.445794,9.783229
4,u1qfjg,52.402592,9.754312,2019,11,7,7,2470.0,1,52.403701,9.754553
...,...,...,...,...,...,...,...,...,...,...,...
5915,u1qfkb9,52.429673,9.703170,2019,11,2,5,1806.0,0,52.429761,9.701390
5916,u1r40s4,52.405308,9.868922,2019,11,3,6,2302.0,0,52.402055,9.870856
5917,u1qfhze,52.424450,9.706125,2019,11,1,5,1499.0,0,52.425862,9.707579
5918,u1qcv70,52.355909,9.724261,2019,7,1,14,165.0,0,52.380353,9.733844


In [24]:
test_non_acc.loc[test_non_acc['cluster_id']==-1]

,geohash,lat,long,UJAHR,UMONAT,UWOCHENTAG,hour,cluster_id,counts,middlelat,middlelong


In [25]:
test_non_acc.to_csv("/data/dadwal/data/DAP_data/dataPrepTrainTestCluster/Baveria/"+city+"/test_nonacc6000.csv",index=False)

In [27]:
def geohasdecode(geo,lat,long):
    if len(geo)==7:
        geo=geo[:-1]
    if lat==0 and long==0:
        lat,long=gh.decode(geo)
    return pd.Series([lat,long])

In [28]:
test_non_acc[['middlelat','middlelong']]=test_non_acc.apply(lambda row: geohasdecode(row['geohash'],row['middlelat'],row['middlelong']), axis=1)

In [29]:
test_non_acc.loc[test_non_acc['middlelat']==0]

,geohash,lat,long,UJAHR,UMONAT,UWOCHENTAG,hour,cluster_id,counts,middlelat,middlelong


In [30]:
test_genertedfinal=test_non_acc[['cluster_id','UJAHR','UMONAT','UWOCHENTAG','hour','middlelat','middlelong','counts']]
test_genertedfinal

,cluster_id,UJAHR,UMONAT,UWOCHENTAG,hour,middlelat,middlelong,counts
0,2014,2019,7,3,12,52.388755,9.806988,1
1,2014,2019,12,5,16,52.388755,9.806988,1
2,2027,2019,8,3,11,52.390804,9.819825,0
3,2027,2019,10,4,7,52.390804,9.819825,0
4,2020,2019,11,1,20,52.387348,9.815376,0
...,...,...,...,...,...,...,...,...
3443,755,2019,11,5,10,52.343185,9.763959,0
3444,900,2019,11,5,10,52.342806,9.771214,0
3445,950,2019,9,3,23,52.345378,9.799214,0
3446,900,2019,8,1,11,52.342806,9.771214,0


In [32]:
def getsolarpos(df,geomap):
    avgliEl=[]
    avgliZe=[]
    avgliAz=[]
    for index, row in df.iterrows():
        liZe=[]
        liAz=[]
        liEl=[]
        year=int(row['UJAHR'])
        month=int(row['UMONAT'])
        day=int(row['UWOCHENTAG'])
        hour=int(row['hour'])
        l=()
        if day==1:
            day=6
        else:
            day=day-2
        A=calendar.TextCalendar(int(day))
        lat=row['middlelat']
        long=row['middlelong']
        for k in A.itermonthdays(year,month):
            if k!=0:
                day1=date(year,month,k)
                if day1.weekday()==day:
                    dtime=str(year)+"-"+str(month).zfill(2)+"-"+str(k).zfill(2)+" "+str(hour).zfill(2)+":"+"00"+":"+"00"
                    date_time_obj = dt.datetime.strptime(dtime, '%Y-%m-%d %H:%M:%S')
                    timezone = pytz.timezone('Europe/Berlin')
                    timezone_date_time_obj = timezone.localize(date_time_obj)
                    az=get_azimuth(lat,long,timezone_date_time_obj)
                    sol_pos=pv.get_solarposition(timezone_date_time_obj,lat,long)
                    getsolarpos=sol_pos[['apparent_zenith', 'zenith', 'apparent_elevation', 'elevation','azimuth', 'equation_of_time']].values
                    ele=getsolarpos[0][3]
                    ze=getsolarpos[0][1]
                    az=getsolarpos[0][4]
                    #print(spl)
                    liEl.append(ele)
                    liZe.append(ze)
                    liAz.append(az)
        npEl=np.array(liEl)
        npZe=np.array(liZe)
        npAz=np.array(liAz)
        
        avgEl=np.mean(liEl)
        avgZe=np.mean(liZe)
        avgAz=np.mean(liAz)

        avgliEl.append(avgEl)
        avgliZe.append(avgZe)
        avgliAz.append(avgAz)
        
    return avgliEl,avgliZe,avgliAz

In [33]:
from multiprocessing import Pool
import numpy as np
def create_sequences(df):
    cores = 20 #Number of CPU cores on your system
    partitions = cores
    pool = Pool(cores)
    
    partition = int(np.ceil(float(len(df)/partitions)))    
    temp = applyParallel (df,getsolarpos,pool,partition,{'geomap':df.copy()})
    retres=temp
    pool.close()
    pool.join()
    elevation = []
    zenith = []
    azimuth=[]
    for set_ in retres:
        elevation.extend(set_[0])
        zenith.extend(set_[1])
        azimuth.extend(set_[2])

    elevation, zenith,azimuth = np.array(elevation), np.array(zenith),np.array(azimuth)   
    return elevation,zenith,azimuth

In [35]:
df_midlle_latlong=test_genertedfinal[['cluster_id','middlelat','middlelong']]
df_midlle_latlong=df_midlle_latlong.drop_duplicates().reset_index().drop(['index'],axis=1)
df_midlle_latlong

,cluster_id,middlelat,middlelong
0,2014,52.388755,9.806988
1,2027,52.390804,9.819825
2,2020,52.387348,9.815376
3,2018,52.390630,9.811047
4,2352,52.381501,9.654190
...,...,...,...
311,940,52.344146,9.784588
312,944,52.343049,9.790043
313,948,52.346671,9.790796
314,941,52.346383,9.777406


In [36]:
train_generted_final=test_genertedfinal.copy()
train_generted_final['is_present']=1
train_generted_final=train_generted_final.sort_values(['cluster_id','UJAHR','UMONAT','UWOCHENTAG','hour'])
train_generted_final=train_generted_final.reset_index(drop=True)
train_generted_final

,cluster_id,UJAHR,UMONAT,UWOCHENTAG,hour,middlelat,middlelong,counts,is_present
0,0,2019,6,3,21,52.351302,9.655861,0,1
1,0,2019,8,1,16,52.351302,9.655861,0,1
2,0,2019,9,1,11,52.340000,9.700000,0,1
3,0,2019,9,6,18,52.351302,9.655861,0,1
4,0,2019,11,1,2,52.350000,9.700000,0,1
...,...,...,...,...,...,...,...,...,...
3443,2493,2019,12,6,9,52.390779,9.779383,0,1
3444,2494,2019,10,3,12,52.405863,9.756617,0,1
3445,2496,2019,9,2,21,52.424502,9.761233,0,1
3446,2496,2019,10,7,0,52.424502,9.761233,0,1


In [37]:
def ispresent(middlelat,middlelong):
    is_present=-1
    if  middlelat==0 and middlelong==0:
        is_present=0
    else:
        is_present=1
    return is_present

In [38]:
import numpy as np
df_final = pd.pivot_table(train_generted_final,
                       index=['cluster_id','UJAHR','UMONAT','UWOCHENTAG','hour'],
                       values=['middlelat', 'middlelong', 'counts'],                            
                       fill_value = 0,
                       dropna=False,
                       aggfunc=np.mean).reset_index()
df_final=df_final.sort_values(['cluster_id','UJAHR','UMONAT','UWOCHENTAG','hour'])
df_final['is_present']=df_final.apply(lambda row: ispresent(row['middlelat'],row['middlelong']), axis=1)
df_final=df_final.reset_index(drop=True)
df_final

,cluster_id,UJAHR,UMONAT,UWOCHENTAG,hour,counts,middlelat,middlelong,is_present
0,0,2019,6,1,0,0.0,0.0,0.0,0
1,0,2019,6,1,1,0.0,0.0,0.0,0
2,0,2019,6,1,2,0.0,0.0,0.0,0
3,0,2019,6,1,3,0.0,0.0,0.0,0
4,0,2019,6,1,4,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...
369259,2498,2019,12,7,19,0.0,0.0,0.0,0
369260,2498,2019,12,7,20,0.0,0.0,0.0,0
369261,2498,2019,12,7,21,0.0,0.0,0.0,0
369262,2498,2019,12,7,22,0.0,0.0,0.0,0


In [39]:
def findpast(row):
    df_filter=pd.DataFrame(columns=['cluster_id', 'UJAHR', 'UMONAT', 'UWOCHENTAG', 'hour', 'middlelat',
       'middlelong', 'counts', 'is_present'])
    df_index = df_final.loc[
      (df_final['UJAHR'] == row['UJAHR']) & (df_final['UMONAT'] == row['UMONAT']) & (
                df_final['cluster_id'] == row['cluster_id'])
       & (df_final['hour'] == row['hour']) & (df_final['UWOCHENTAG'] == row['UWOCHENTAG'])
    &(df_final['is_present'] == row['is_present'])].index[0]
    i=int(df_index)-8
    for i in range(i,df_index):
         df_filter=df_filter.append((df_final.iloc[[i]]),ignore_index=True)        
    return df_filter

In [40]:
def getdata(df_1,geomap):
    df=pd.DataFrame()
    for index, row in df_1.iterrows(): 
        temp=findpast(row)
        temp['is_p2']=0
        df=df.append(temp)
        temp1=row
        temp1['is_p2']=2
        df=df.append(temp1)
        
    return df
                                                               

In [41]:
from multiprocessing import Pool
import numpy as np
def create_sequences_1(df):
    cores = 20#Number of CPU cores on your system
    partitions = cores
    pool = Pool(cores)
    
    partition = int(np.ceil(float(len(df)/partitions)))    
    temp = applyParallel (train_generted_final,getdata,pool,partition,{'geomap':df.copy()})
    retres=temp
    pool.close()
    pool.join()
    final = []
    final.extend(retres)
    return final

In [42]:
df=create_sequences_1(train_generted_final)
df

<class 'list'>
[    cluster_id UJAHR UMONAT UWOCHENTAG hour  middlelat  middlelong  counts  \
0            0  2019      6          3   13    0.00000    0.000000     0.0   
1            0  2019      6          3   14    0.00000    0.000000     0.0   
2            0  2019      6          3   15    0.00000    0.000000     0.0   
3            0  2019      6          3   16    0.00000    0.000000     0.0   
4            0  2019      6          3   17    0.00000    0.000000     0.0   
..         ...   ...    ...        ...  ...        ...         ...     ...   
4           57  2019      7          3   23    0.00000    0.000000     0.0   
5           57  2019      7          4    0    0.00000    0.000000     0.0   
6           57  2019      7          4    1    0.00000    0.000000     0.0   
7           57  2019      7          4    2    0.00000    0.000000     0.0   
172         57  2019      7          4    3   52.34337    9.719863     0.0   

    is_present  is_p2  
0            0    0.0  

[    cluster_id UJAHR UMONAT UWOCHENTAG hour  middlelat  middlelong  counts  \
 0            0  2019      6          3   13    0.00000    0.000000     0.0   
 1            0  2019      6          3   14    0.00000    0.000000     0.0   
 2            0  2019      6          3   15    0.00000    0.000000     0.0   
 3            0  2019      6          3   16    0.00000    0.000000     0.0   
 4            0  2019      6          3   17    0.00000    0.000000     0.0   
 ..         ...   ...    ...        ...  ...        ...         ...     ...   
 4           57  2019      7          3   23    0.00000    0.000000     0.0   
 5           57  2019      7          4    0    0.00000    0.000000     0.0   
 6           57  2019      7          4    1    0.00000    0.000000     0.0   
 7           57  2019      7          4    2    0.00000    0.000000     0.0   
 172         57  2019      7          4    3   52.34337    9.719863     0.0   
 
     is_present  is_p2  
 0            0    0.0  


In [43]:
df_final1=pd.DataFrame()
for df1 in df:
    df_final1=df_final1.append(df1)
df_final1=df_final1.reset_index(drop=True)
df_final1

,cluster_id,UJAHR,UMONAT,UWOCHENTAG,hour,middlelat,middlelong,counts,is_present,is_p2
0,0,2019,6,3,13,0.000000,0.000000,0.0,0,0.0
1,0,2019,6,3,14,0.000000,0.000000,0.0,0,0.0
2,0,2019,6,3,15,0.000000,0.000000,0.0,0,0.0
3,0,2019,6,3,16,0.000000,0.000000,0.0,0,0.0
4,0,2019,6,3,17,0.000000,0.000000,0.0,0,0.0
...,...,...,...,...,...,...,...,...,...,...
31027,2498,2019,6,3,13,0.000000,0.000000,0.0,0,0.0
31028,2498,2019,6,3,14,0.000000,0.000000,0.0,0,0.0
31029,2498,2019,6,3,15,0.000000,0.000000,0.0,0,0.0
31030,2498,2019,6,3,16,0.000000,0.000000,0.0,0,0.0


In [46]:
join_middle_latlong=pd.merge(df_final1,df_midlle_latlong,on='cluster_id',how='left')
join_middle_latlong.drop(['middlelat_x','middlelong_x'],axis=1,inplace=True)
join_middle_latlong.columns=['cluster_id', 'UJAHR', 'UMONAT', 'UWOCHENTAG', 'hour',
       'counts','is_present','is_p2', 'middlelat', 'middlelong']
join_middle_latlong

,cluster_id,UJAHR,UMONAT,UWOCHENTAG,hour,counts,is_present,is_p2,middlelat,middlelong
0,0,2019,6,3,13,0.0,0,0.0,52.340000,9.700000
1,0,2019,6,3,13,0.0,0,0.0,52.350000,9.700000
2,0,2019,6,3,13,0.0,0,0.0,52.351302,9.655861
3,0,2019,6,3,14,0.0,0,0.0,52.340000,9.700000
4,0,2019,6,3,14,0.0,0,0.0,52.350000,9.700000
...,...,...,...,...,...,...,...,...,...,...
31135,2498,2019,6,3,13,0.0,0,0.0,52.422463,9.770763
31136,2498,2019,6,3,14,0.0,0,0.0,52.422463,9.770763
31137,2498,2019,6,3,15,0.0,0,0.0,52.422463,9.770763
31138,2498,2019,6,3,16,0.0,0,0.0,52.422463,9.770763


In [49]:
elevation,zenith,azimuth=create_sequences(join_middle_latlong)

/usr/local/lib/python3.7/dist-packages/pysolar/solartime.py:112: UserWarning: I don't know about leap seconds after 2018
  (leap_seconds_base_year + len(leap_seconds_adjustments) - 1)
/usr/local/lib/python3.7/dist-packages/pysolar/solartime.py:112: UserWarning: I don't know about leap seconds after 2018
  (leap_seconds_base_year + len(leap_seconds_adjustments) - 1)
/usr/local/lib/python3.7/dist-packages/pysolar/solartime.py:112: UserWarning: I don't know about leap seconds after 2018
  (leap_seconds_base_year + len(leap_seconds_adjustments) - 1)
/usr/local/lib/python3.7/dist-packages/pysolar/solartime.py:112: UserWarning: I don't know about leap seconds after 2018
  (leap_seconds_base_year + len(leap_seconds_adjustments) - 1)
/usr/local/lib/python3.7/dist-packages/pysolar/solartime.py:112: UserWarning: I don't know about leap seconds after 2018
  (leap_seconds_base_year + len(leap_seconds_adjustments) - 1)
/usr/local/lib/python3.7/dist-packages/pysolar/solartime.py:112: UserWarning: I 

[([60.43807980830446, 60.42823578959958, 60.42220300804477, 59.822402472770634, 59.81287932992922, 59.819844965479106, 55.22064174158564, 55.213229345626985, 55.23034870134117, 48.0230393408055, 48.01822825938574, 48.04123092948058, 39.49156824018186, 39.489203373630836, 39.51508919080076, 30.419815366021545, 30.419647521608155, 30.446582878962204, 21.306115816143205, 21.307949659395977, 21.33469391525357, 12.519036212741042, 12.522732538463597, 12.548268412502317, 4.39087834227143, 4.39631994119664, 4.4196524839018405, 16.389482924435775, 16.389847402153304, 16.36296629280523, 25.501943332181746, 25.500198427983324, 25.473438367422013, 34.24947966520593, 34.24549878107993, 34.22026267295238, 42.06101717562652, 42.05471187061623, 42.03297902891721, 48.12526619928818, 48.116796596346184, 48.10137048785965, 51.4175198964517, 51.407673533885706, 51.40168676756791, 51.13415722374704, 51.12442842518861, 51.12938538462989, 47.34955904547373, 47.34137101453307, 47.35576634398142, 40.945692854

In [51]:
join_middle_latlong['elevaion']=elevation
join_middle_latlong['zenith']=zenith
join_middle_latlong['azimuth_solar']=azimuth
join_middle_latlong

,cluster_id,UJAHR,UMONAT,UWOCHENTAG,hour,counts,is_present,is_p2,middlelat,middlelong,elevaion,zenith,azimuth_solar
0,0,2019,6,3,13,0.0,0,0.0,52.340000,9.700000,60.438080,29.561920,169.887068
1,0,2019,6,3,13,0.0,0,0.0,52.350000,9.700000,60.428236,29.571764,169.890166
2,0,2019,6,3,13,0.0,0,0.0,52.351302,9.655861,60.422203,29.577797,169.808855
3,0,2019,6,3,14,0.0,0,0.0,52.340000,9.700000,59.822402,30.177598,197.765143
4,0,2019,6,3,14,0.0,0,0.0,52.350000,9.700000,59.812879,30.187121,197.759899
...,...,...,...,...,...,...,...,...,...,...,...,...,...
31135,2498,2019,6,3,13,0.0,0,0.0,52.422463,9.770763,60.364410,29.635590,170.043323
31136,2498,2019,6,3,14,0.0,0,0.0,52.422463,9.770763,59.730683,30.269317,197.848530
31137,2498,2019,6,3,15,0.0,0,0.0,52.422463,9.770763,55.130529,34.869471,222.191254
31138,2498,2019,6,3,16,0.0,0,0.0,52.422463,9.770763,47.945502,42.054498,241.246687


In [53]:
join_middle_latlong['concate_year']='20172018'
join_middle_latlong.columns

Index(['cluster_id', 'UJAHR', 'UMONAT', 'UWOCHENTAG', 'hour', 'counts',
       'is_present', 'is_p2', 'middlelat', 'middlelong', 'elevaion', 'zenith',
       'azimuth_solar', 'concate_year'],
      dtype='object')

In [55]:
join_middle_latlong.to_csv("/data/dadwal/data/DAP_data/dataPrepTrainTestCluster/Baveria/"+city+"/test_final_withSolarConvareyear6000.csv",index=False)

In [56]:
l=[]
year=[]
i=0
parts1=[]
with open("/data/dadwal/data/DAP_data/dataPrepTrainTestCluster/Baveria/"+city+"/test_final_withSolarConvareyear6000.csv", 'r') as file:
    header = False
    for line in file:
        if not header:
            header = True
            continue
        parts = line.replace('\r', '').replace('\n', '').split(',')
        if [parts[0],parts[13]] not in l:
            m=[parts[0],parts[13]]
            l.append(m)
            i=0
            parts.append(i)
        else:
            i=i+1
            parts.append(i) 
        parts1.append(parts)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [57]:
import csv

def writeCsvFile(fname, data, *args, **kwargs):
    mycsv = csv.writer(open(fname, 'w'), *args, **kwargs)
    for row in data:
        mycsv.writerow(row)

writeCsvFile(r"/data/dadwal/data/DAP_data/dataPrepTrainTestCluster/Baveria/"+city+"/testdatawithazimuthUTPYE3OhannwithIndex6000.csv", parts1)

In [58]:
import pandas as pd
hann_indexed=pd.read_csv('/data/dadwal/data/DAP_data/dataPrepTrainTestCluster/Baveria/'+city+'/testdatawithazimuthUTPYE3OhannwithIndex6000.csv')

hann_indexed.columns=['cluster_id', 'UJAHR', 'UMONAT', 'UWOCHENTAG', 'hour', 'counts',
       'is_present', 'is_p2', 'middlelat', 'middlelong', 'elevaion', 'zenith',
       'azimuth_solar', 'concate_year', 'timestep']
hann_indexed.to_csv('/data/dadwal/data/DAP_data/dataPrepTrainTestCluster/Baveria/'+city+'/testdatawithazimuthUTPYE3OhannwithIndex6000.csv',index=False)

hann_indexed

,cluster_id,UJAHR,UMONAT,UWOCHENTAG,hour,counts,is_present,is_p2,middlelat,middlelong,elevaion,zenith,azimuth_solar,concate_year,timestep
0,0.0,2019.0,6.0,3.0,13.0,0.0,0.0,0.0,52.350000,9.700000,60.428236,29.571764,169.890166,20172018,1
1,0.0,2019.0,6.0,3.0,13.0,0.0,0.0,0.0,52.351302,9.655861,60.422203,29.577797,169.808855,20172018,2
2,0.0,2019.0,6.0,3.0,14.0,0.0,0.0,0.0,52.340000,9.700000,59.822402,30.177598,197.765143,20172018,3
3,0.0,2019.0,6.0,3.0,14.0,0.0,0.0,0.0,52.350000,9.700000,59.812879,30.187121,197.759899,20172018,4
4,0.0,2019.0,6.0,3.0,14.0,0.0,0.0,0.0,52.351302,9.655861,59.819845,30.180155,197.680102,20172018,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31134,2498.0,2019.0,6.0,3.0,13.0,0.0,0.0,0.0,52.422463,9.770763,60.364410,29.635590,170.043323,20172018,4
31135,2498.0,2019.0,6.0,3.0,14.0,0.0,0.0,0.0,52.422463,9.770763,59.730683,30.269317,197.848530,20172018,5
31136,2498.0,2019.0,6.0,3.0,15.0,0.0,0.0,0.0,52.422463,9.770763,55.130529,34.869471,222.191254,20172018,6
31137,2498.0,2019.0,6.0,3.0,16.0,0.0,0.0,0.0,52.422463,9.770763,47.945502,42.054498,241.246687,20172018,7


In [61]:
# creating h5 file

In [62]:
import pandas as pd
import numpy as np

import random
import os
import sys
import psutil

import matplotlib
import matplotlib.pyplot as plt
import math
from multiprocessing import cpu_count,Pool 
import multiprocessing

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

from sklearn.preprocessing import MinMaxScaler
import pickle


In [63]:
def daynight(month,hour):
    if hour>=22 or (hour>=0 and hour<4):
        return 0
    elif hour>=8 and hour<=15:
        return 1
    if month==1:
        if (hour>=4 and hour<=8) or (hour>=16 and hour<=22):
            return 0
        else:
            return 1
    if month==2:
        if (hour>=4 and hour<=8) or (hour>=18 and hour<=22):
            return 0
        if hour>=16 and hour <=18:
            return 1
    if month==3:
        if (hour>=4 and hour<=6) or  (hour>=19 and hour<=22):
            return 0
        if (hour>= 7 or hour <=8) or (hour>=16 and hour <=18):
            return 1
    if month==4:
        if (hour>=4 and hour<=5) or (hour>=20 and hour<=22):
            return 0
        if (hour>= 6 and hour <=8) or (hour>= 16 and hour <=20):
            return 1
    if month==5 or month==6 or month==7 or month==8:
        if (hour>=4 and hour<=5) or (hour==22):
            return 0
        if (hour>=6 and hour<=8) or (hour>=16 and hour<=21):
            return 1 
    if month==9 or month==10:
        if (hour>=4 and hour<=7 )or (hour>=20 and hour<=22):
            return 0
        if  (hour>=16 and hour<=19) :
            return 1 
    if month==11 or month==12:
        if(hour>=4 and hour<=8) or (hour>=16 and hour<=22):
            return 0 
        
                 

In [64]:
def time_interval_two(HOD):        
    if HOD>=6 and HOD<=10 :
        return 0
    if  HOD>=11 and HOD<=15 :
        return 1
    if  HOD>=16 and HOD<=17:
        return 2;
    if HOD>=18 and HOD<=22:
        return 3
    if HOD>22 or (HOD>=0 and HOD<6):
        return 4
    else:
        return 5;

In [65]:
#Officially spring is during the months of March, April and May in Germany. Summer is from June through to August. 
#Autumn is during the months of September, October and November and winter is from December to February. 
def season(month):
    if month>=3 and month<=5:
        return 0
    if month>=6 and month<=8:
        return 1
    if month>=9 and month<=11:
        return 2
    if month==12 or month==1 or month== 2:
        return 3
    

In [66]:
f = open('/data/dadwal/data/DAP_data/dataPrepTrainTestCluster/Baveria/'+city+'/geodata/geo_dict.pkl', "rb")
geo_dict = pickle.load(f)
f.close()

train_geodict=pd.DataFrame(geo_dict.items(), columns=['cluster_id', 'value'])
train_geodict

,cluster_id,value
0,0.0,0
1,2.0,1
2,3.0,2
3,5.0,3
4,9.0,4
...,...,...
356,2493.0,356
357,2494.0,357
358,2495.0,358
359,2496.0,359


In [67]:

geo_dict1 = dict(zip(sorted(hann_indexed.cluster_id.unique()), range(len(hann_indexed.cluster_id.unique()))))
print(geo_dict1)
test_geodict=pd.DataFrame(geo_dict1.items(), columns=['cluster_id', 'value'])
test_geodict

{0.0: 0, 2.0: 1, 3.0: 2, 5.0: 3, 9.0: 4, 10.0: 5, 11.0: 6, 19.0: 7, 24.0: 8, 28.0: 9, 31.0: 10, 32.0: 11, 33.0: 12, 34.0: 13, 35.0: 14, 36.0: 15, 42.0: 16, 47.0: 17, 50.0: 18, 54.0: 19, 57.0: 20, 61.0: 21, 63.0: 22, 75.0: 23, 81.0: 24, 82.0: 25, 83.0: 26, 84.0: 27, 85.0: 28, 87.0: 29, 92.0: 30, 93.0: 31, 94.0: 32, 97.0: 33, 98.0: 34, 100.0: 35, 103.0: 36, 104.0: 37, 105.0: 38, 106.0: 39, 115.0: 40, 121.0: 41, 122.0: 42, 130.0: 43, 134.0: 44, 138.0: 45, 140.0: 46, 141.0: 47, 142.0: 48, 145.0: 49, 147.0: 50, 153.0: 51, 155.0: 52, 158.0: 53, 160.0: 54, 161.0: 55, 164.0: 56, 165.0: 57, 170.0: 58, 172.0: 59, 173.0: 60, 176.0: 61, 178.0: 62, 182.0: 63, 184.0: 64, 190.0: 65, 191.0: 66, 193.0: 67, 198.0: 68, 202.0: 69, 203.0: 70, 204.0: 71, 234.0: 72, 269.0: 73, 279.0: 74, 283.0: 75, 289.0: 76, 309.0: 77, 311.0: 78, 342.0: 79, 357.0: 80, 360.0: 81, 394.0: 82, 395.0: 83, 396.0: 84, 398.0: 85, 402.0: 86, 405.0: 87, 486.0: 88, 741.0: 89, 743.0: 90, 745.0: 91, 746.0: 92, 747.0: 93, 755.0: 94, 756.

,cluster_id,value
0,0.0,0
1,2.0,1
2,3.0,2
3,5.0,3
4,9.0,4
...,...,...
309,2492.0,309
310,2493.0,310
311,2494.0,311
312,2496.0,312


In [68]:
merge_geodict=pd.concat([train_geodict[['cluster_id']],test_geodict[['cluster_id']]])
merge_geodict=merge_geodict.drop_duplicates('cluster_id')

merge_geodict=merge_geodict.reset_index(drop=True)
merge_geodict

,cluster_id
0,0.0
1,2.0
2,3.0
3,5.0
4,9.0
...,...
356,2493.0
357,2494.0
358,2495.0
359,2496.0


In [69]:
geo_dict = dict(zip(sorted(merge_geodict.cluster_id.unique()), range(len(merge_geodict.cluster_id.unique()))))
print(geo_dict)
f = open("/data/dadwal/data/DAP_data/dataPrepTrainTestCluster/Baveria/"+city+"/geodata/geo_dict.pkl","wb")
pickle.dump(geo_dict,f)


{0.0: 0, 2.0: 1, 3.0: 2, 5.0: 3, 9.0: 4, 10.0: 5, 11.0: 6, 19.0: 7, 24.0: 8, 28.0: 9, 31.0: 10, 32.0: 11, 33.0: 12, 34.0: 13, 35.0: 14, 36.0: 15, 42.0: 16, 47.0: 17, 50.0: 18, 54.0: 19, 57.0: 20, 59.0: 21, 61.0: 22, 63.0: 23, 75.0: 24, 81.0: 25, 82.0: 26, 83.0: 27, 84.0: 28, 85.0: 29, 87.0: 30, 92.0: 31, 93.0: 32, 94.0: 33, 97.0: 34, 98.0: 35, 100.0: 36, 103.0: 37, 104.0: 38, 105.0: 39, 106.0: 40, 115.0: 41, 121.0: 42, 122.0: 43, 130.0: 44, 134.0: 45, 138.0: 46, 140.0: 47, 141.0: 48, 142.0: 49, 145.0: 50, 147.0: 51, 151.0: 52, 153.0: 53, 155.0: 54, 158.0: 55, 160.0: 56, 161.0: 57, 164.0: 58, 165.0: 59, 170.0: 60, 172.0: 61, 173.0: 62, 176.0: 63, 178.0: 64, 182.0: 65, 184.0: 66, 190.0: 67, 191.0: 68, 192.0: 69, 193.0: 70, 198.0: 71, 202.0: 72, 203.0: 73, 204.0: 74, 234.0: 75, 269.0: 76, 279.0: 77, 283.0: 78, 289.0: 79, 308.0: 80, 309.0: 81, 311.0: 82, 342.0: 83, 357.0: 84, 360.0: 85, 394.0: 86, 395.0: 87, 396.0: 88, 398.0: 89, 402.0: 90, 405.0: 91, 486.0: 92, 741.0: 93, 743.0: 94, 745.0

In [71]:
hann_indexed.columns=['geohash','year', 'month', 'day', 'twohoursession',
       'acc_count', 'is_present','is_p2', 'middlelat', 'middlelong', 'elevaion',
       'zenith', 'azimuth_solar', 'concate_year', 'timestep']
hann_indexed.to_csv("/data/dadwal/data/DAP_data/dataPrepTrainTestCluster/Baveria/"+city+"/df_finalTest6000.csv",index=False)

In [72]:
def create_temporaldata(filepath,city):
    df = pd.read_csv(filepath)
    f = open("/data/dadwal/data/DAP_data/dataPrepTrainTestCluster/Baveria/"+city+"/geo_dict.pkl", "rb")
    geo_dict = pickle.load(f)
    f.close()
    def fun_hash(geohash):
        return geo_dict[geohash]

    df['geohash_code'] = df.apply(lambda row: fun_hash(row['geohash']), axis=1)
    def week_day(DOW):
        if DOW > 1 and DOW <= 6:
            return 1
        elif DOW == 1 or DOW == 7:
            return 0

    def make_binary(d):
        if d > 0:
            return 1
        else:
            return 0

    def year_ternary(d):
        if d == 2017:
            return 0
        if d==2018:
            return 1
        else:
            return 2

    df['DOW_cat'] = df.apply(lambda row: week_day(row['day']), axis=1)
    df['HOD_cat'] = df.apply(lambda row: time_interval_two(row['twohoursession']), axis=1)
    df['DayLight'] = df.apply(lambda row: daynight(row['month'], row['twohoursession']), axis=1)
    df['season'] = df.apply(lambda row: season(row['month']), axis=1)
    df['year'] = df.apply(lambda row: year_ternary(row['year']), axis=1)
    df['predicted_accident']=df.apply(lambda row: make_binary(row['acc_count']), axis=1)
    mergespeed_tempdf = df[[u'timestep', u'predicted_accident',u'is_p2',u'is_present', u'geohash', u'geohash_code',u'year', u'month', u'day',u'twohoursession', 
                  u'HOD_cat', u'DOW_cat', u'DayLight', u'season', u'elevaion', u'zenith', u'azimuth_solar', 'acc_count'
                           ]]
    mergespeed_tempdf.to_hdf("/data/dadwal/data/DAP_data/dataPrepTrainTestCluster/Baveria/"+city+"/"+"ng_test.h5",
                   key='set3')
    display(mergespeed_tempdf.head())


In [73]:
print(city)

new_method/hann/clustering/gridgrowing6


In [75]:
create_temporaldata('/data/dadwal/data/DAP_data/dataPrepTrainTestCluster/Baveria/'+city+'/df_finalTest6000.csv', city)

,geohash,year,month,day,twohoursession,acc_count,is_present,is_p2,middlelat,middlelong,elevaion,zenith,azimuth_solar,concate_year,timestep
0,0.0,2019.0,6.0,3.0,13.0,0.0,0.0,0.0,52.350000,9.700000,60.428236,29.571764,169.890166,20172018,1
1,0.0,2019.0,6.0,3.0,13.0,0.0,0.0,0.0,52.351302,9.655861,60.422203,29.577797,169.808855,20172018,2
2,0.0,2019.0,6.0,3.0,14.0,0.0,0.0,0.0,52.340000,9.700000,59.822402,30.177598,197.765143,20172018,3
3,0.0,2019.0,6.0,3.0,14.0,0.0,0.0,0.0,52.350000,9.700000,59.812879,30.187121,197.759899,20172018,4
4,0.0,2019.0,6.0,3.0,14.0,0.0,0.0,0.0,52.351302,9.655861,59.819845,30.180155,197.680102,20172018,5


zero accident = 0.8269372812229038


,timestep,predicted_accident,is_p2,is_present,geohash,geohash_code,year,month,day,twohoursession,HOD_cat,DOW_cat,DayLight,season,elevaion,zenith,azimuth_solar,acc_count,middlelat,middlelong
0,1,0,0.0,0.0,0.0,0,2,6.0,3.0,13.0,1,1,1,1,60.428236,29.571764,169.890166,0.0,52.350000,9.700000
1,2,0,0.0,0.0,0.0,0,2,6.0,3.0,13.0,1,1,1,1,60.422203,29.577797,169.808855,0.0,52.351302,9.655861
2,3,0,0.0,0.0,0.0,0,2,6.0,3.0,14.0,1,1,1,1,59.822402,30.177598,197.765143,0.0,52.340000,9.700000
3,4,0,0.0,0.0,0.0,0,2,6.0,3.0,14.0,1,1,1,1,59.812879,30.187121,197.759899,0.0,52.350000,9.700000
4,5,0,0.0,0.0,0.0,0,2,6.0,3.0,14.0,1,1,1,1,59.819845,30.180155,197.680102,0.0,52.351302,9.655861


# train and test

In [76]:
import pandas as pd
import numpy as np

import random
import os
import sys
import psutil

import matplotlib
import matplotlib.pyplot as plt
import math
from multiprocessing import cpu_count,Pool 
import multiprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from collections import Counter

import pickle


In [77]:
print(city)


new_method/hann/clustering/gridgrowing6


In [78]:
f=open("/data/dadwal/data/DAP_data/dataPrepTrainTestCluster/Baveria/"+city+"//geodata//clustered_data_poi_acc_merged.pkl","rb")
geohash_dict = pickle.load(f)
f.close()
print(geohash_dict)

f=open("/data/dadwal/data/DAP_data/dataPrepTrainTestCluster/Baveria/"+city+"/geodata/geo_dict.pkl","rb")
geo_dict = pickle.load(f)
f.close()
#print(geo_dict)

f=open("/data/dadwal/data/DAP_data/dataPrepTrainTestCluster/Baveria/"+city+"//geodata//uartTrainOnly.pkl","rb")
NLP_dict_uart = pickle.load(f)
f.close()
#print(geo_dict)

f = open("/data/dadwal/data/DAP_data/dataPrepTrainTestCluster/Baveria/"+city+"//geodata//zustandTrainOnly.pkl","rb")
NLP_dict_zustand = pickle.load(f)
f.close()

f = open("/data/dadwal/data/DAP_data/dataPrepTrainTestCluster/Baveria/"+city+"//geodata//utypeTrainOnly.pkl","rb")
NLP_dict_utype1 = pickle.load(f)
f.close()


f = open("/data/dadwal/data/DAP_data/dataPrepTrainTestCluster/Baveria/"+city+"///geodata//acc_countTrainOnly.pkl","rb")
NLP_dict_accCount = pickle.load(f)
f.close()
#print((NLP_dict_accCount))


f = open("/data/dadwal/data/DAP_data/dataPrepTrainTestCluster/Baveria/"+city+"//geodata//htype.pkl","rb")
NLP_dict_htype = pickle.load(f)
f.close()

f = open("/data/dadwal/data/DAP_data/dataPrepTrainTestCluster/Baveria/"+city+"//geodata//maxspeed.pkl","rb")
NLP_dict_avg_max_speed = pickle.load(f)
f.close()

f = open("/data/dadwal/data/DAP_data/dataPrepTrainTestCluster/Baveria/"+city+"//geodata/hann_centerCord.pkl","rb")
NLP_dict_centerCord = pickle.load(f)
f.close()

f = open("/data/dadwal/data/DAP_data/dataPrepTrainTestCluster/Baveria/"+city+"//geodata/clusterToGeohashCountTrainOnly.pkl","rb")
NLP_dict_clustertoGeohash = pickle.load(f)
f.close()



{0.0: array([0.00087002, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.00818793]), 2.0: array([0.00116003, 0.        , 0.        , 0.        , 0.00200803,
       0.        , 0.        , 0.        , 0.        , 0.00590701]), 3.0: array([0.000174  , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.00608246]), 5.0: array([0.00052201, 0.        , 0.        , 0.        , 0.00133869,
       0.        , 0.        , 0.        , 0.        , 0.00807096]), 9.0: array([0.00023201, 0.        , 0.        , 0.00012771, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.00120869]), 10.0: array([0.000116  , 0.        , 0.        , 0.        , 0.00334672,
       0.        , 0.        , 0.        , 0.        , 0.00241739]), 11.0: array([0.00510411, 0.        , 0.        , 0.01877395, 0.03748327,
       0.        , 0.00709579, 0.        , 0.        , 0.01329564]), 19.0: arra

In [79]:
cores = cpu_count() #Number of CPU cores on your system
partitions = 25

class WithExtraArgs(object):
     def __init__(self, func, **args):
         self.func = func
         self.args = args
     def __call__(self, df):
         return self.func(df, **self.args)

def applyParallel(data, func,pool,partition, kwargs):
    data_split = [data[i:i + partition] for i in range(0, len(data), partition)]
    data =pool.map(WithExtraArgs(func, **kwargs), data_split)
    return data

In [80]:
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn import preprocessing

In [81]:
def onhot_enoceder_month(train):
    mlb = preprocessing.MultiLabelBinarizer(classes=(1,2,3,4,5,6,7,8,9,10,11,12))
    pf=train['month'].values.reshape(-1, 1)#.tolist()
    k = mlb.fit_transform((pf))
    onehot_encode = pd.concat([train.reset_index().drop('month',1),
             pd.DataFrame( mlb.fit_transform((pf))
                           )], axis=1)#.reindex()
    return onehot_encode.drop('index',1)

In [82]:
from sklearn import preprocessing
def onhot_enoceder_day(train):
    mlb = preprocessing.MultiLabelBinarizer(classes=(1,2,3,4,5,6,7))
    pf=train['day'].values.reshape(-1, 1)#.tolist()
    k = mlb.fit_transform((pf))
    onehot_encode = pd.concat([train.reset_index().drop('day',1),
             pd.DataFrame( mlb.fit_transform((pf))
                          #columns=['m_en0','m_en1','m_en2','m_en3','m_en4'])], axis=1).reindex()
                          )], axis=1)#.reindex()
    return onehot_encode.drop('index',1)

In [83]:
def onhot_enoceder(train):
    mlb = preprocessing.MultiLabelBinarizer(classes=(1,2,3,4,5))
    pf=train['HOD_cat'].values.reshape(-1, 1).tolist()
    onehot_encode = pd.concat([train.reset_index().drop('HOD_cat',1),
                 pd.DataFrame(mlb.fit_transform((pf))
                              )], axis=1).reindex()
    return onehot_encode.drop('index',1)

In [84]:
def scalarval(df):
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaler.fit(df.values.reshape(-1,1)) 
    scaled_values = scaler.transform(df.values.reshape(-1,1)) 
    df = scaled_values
    return df

In [85]:
def onhot_enoceder_season(train):
    myEncoder = OneHotEncoder(sparse=False)
    mlb = preprocessing.MultiLabelBinarizer(classes=(1,2,3,4))
    pf=train['season'].values.reshape(-1, 1).tolist()

    onehot_encode = pd.concat([train.reset_index().drop('season',1),
                 pd.DataFrame(mlb.fit_transform((pf))
                              )], axis=1).reindex()
    return onehot_encode.drop('index',1)

In [86]:
def onhot_enoceder_year(train):
    myEncoder = OneHotEncoder(sparse=False)
    mlb = preprocessing.MultiLabelBinarizer(classes=(1,2,3))
    pf=train['year'].values.reshape(-1, 1).tolist()

    onehot_encode = pd.concat([train.reset_index().drop('year',1),
                 pd.DataFrame(mlb.fit_transform((pf))
                              )], axis=1).reindex()
    return onehot_encode.drop('index',1)

In [87]:
def create_train_set_aug_geo(frame_list,geomap):
    process_name = str(multiprocessing.current_process())
    id = int(process_name.split(',')[0].split('-')[1])
    X_train = []
    y_train = []
    for frame in frame_list:
        training_set = frame.values
        geo_code = geo_dict[frame.geohash.iloc[0]]
        htype_count = 0
        for key, values in NLP_dict_htype.items():
            htype_count = len(values)
            break
        
        try:
            NLP_center_cord=NLP_dict_centerCord[frame.geohash.iloc[0]]
        except:
            NLP_center_cord = np.zeros(2) 
        try:
            NLP_cluster_geohash=NLP_dict_clustertoGeohash[frame.geohash.iloc[0]]
        except:
            NLP_cluster_geohash = np.zeros(1) 

        
        try:
            NLP_htype = NLP_dict_htype[frame.geohash.iloc[0]]
        except:
            NLP_htype = np.zeros(htype_count)
        try:
            geo_vec = geomap[frame.geohash.iloc[0]]
        except:
            geo_vec = np.zeros(10) 

        try:
            NLP_code_uart = NLP_dict_uart[frame.geohash.iloc[0]]
        except:
            NLP_code_uart = np.zeros(10) 

        try:
            NLP_code_uytpe1 = NLP_dict_utype1[frame.geohash.iloc[0]]
        except:
            NLP_code_uytpe1 = np.zeros(7) 

        try:
            NLP_code_zustand = NLP_dict_zustand[frame.geohash.iloc[0]]
        except:
            NLP_code_zustand = np.zeros(3) 
        
        try:
            NLP_code_max_speed = NLP_dict_avg_max_speed[frame.geohash.iloc[0]]
        except:
            NLP_code_max_speed=[0.24]
             
        try:
            NLP_code_acc_count = NLP_dict_accCount[frame.geohash.iloc[0]]
        except:
            #print('raj')
            NLP_code_acc_count = np.zeros(1) 

        
        for i in range(8, training_set.shape[0]):
            if training_set[i, 2] > 1 : 
                if training_set[i, 1] > 0:
                    a = np.concatenate((training_set[i-8:i,7:].flatten(),geo_vec),axis=0)
                    a = np.concatenate((a,NLP_code_uart),axis=0)
                    a= np.concatenate((a,NLP_code_uytpe1),axis=0)
                    a=np.concatenate((a,NLP_code_zustand),axis=0)
                    a=np.concatenate((a,NLP_code_acc_count),axis=0)
                    a=np.concatenate((a,NLP_code_max_speed),axis=0)
                    a=np.concatenate((a,NLP_htype),axis=0)
                    a=np.concatenate((a,NLP_center_cord),axis=0)
                    a=np.concatenate((a,NLP_cluster_geohash),axis=0)
                    a = np.append(a, geo_code)
                    X_train.append(a)
                    y_train.append(1)
                else:
                    a = np.concatenate((training_set[i-8:i,7:].flatten(),geo_vec),axis=0)
                    a = np.concatenate((a,NLP_code_uart),axis=0)
                    a= np.concatenate((a,NLP_code_uytpe1),axis=0)
                    a=np.concatenate((a,NLP_code_zustand),axis=0)
                    a=np.concatenate((a,NLP_code_acc_count),axis=0)
                    a=np.concatenate((a,NLP_code_max_speed),axis=0)
                    a=np.concatenate((a,NLP_htype),axis=0)
                    a=np.concatenate((a,NLP_center_cord),axis=0)
                    a=np.concatenate((a,NLP_cluster_geohash),axis=0)
                    a = np.append(a, geo_code)
                    X_train.append(a)
                    y_train.append(0)
    return X_train, y_train

In [88]:
def create_sequences(df,geohash_dict):
    frame_list=[]
    for idx, frame in df.groupby(df.geohash):        
        frame_list.append(frame)
    pool = Pool(cores)
    partition = int(np.ceil(float(len(frame_list))/partitions))    
    train_set = applyParallel (frame_list,create_train_set_aug_geo,pool,partition,{'geomap':geohash_dict.copy()})
    pool.close()
    pool.join()
    X_train = []
    y_train = []
    for set_ in train_set:
        X_train.extend(set_[0])
        y_train.extend(set_[1])

    X_train, y_train = np.array(X_train), np.array(y_train)   
    return X_train,y_train

In [89]:
print(city)

new_method/hann/clustering/gridgrowing6


In [92]:
def train_data(filename):
    df=pd.read_hdf('/data/dadwal/data/DAP_data/dataPrepTrainTestCluster/Baveria/'+city+'/'+'ng_val6000.h5',key='set3')    
    df=df[[u'timestep', u'predicted_accident',u'is_p2',u'is_present', u'geohash', u'geohash_code', u'month',u'day', u'twohoursession',u'year', 
                  u'HOD_cat', u'DOW_cat', u'DayLight', u'season', u'elevaion', u'zenith', u'azimuth_solar'
                           ]]
    display(df.head())
    df = onhot_enoceder_month(df)
    df= onhot_enoceder_day(df)
    df=onhot_enoceder(df)
    df=onhot_enoceder_season(df)
    df=onhot_enoceder_year(df)
    df['elevaion'] = scalarval(df['elevaion'])
    df['zenith'] = scalarval(df['zenith'])
    df['azimuth_solar'] = scalarval(df['azimuth_solar'])
    df['previous_acc']=df['predicted_accident']
    print(df.columns)
    df_normalize = df.copy()
    
    X_train, y_train = create_sequences(df,geohash_dict)
 
    np.save('/data/dadwal/data/DAP_data/dataPrepTrainTestCluster/Baveria/'+city+'/traindata/X_val6000',X_train) 
    print (X_train.shape)
    np.save('/data/dadwal/data/DAP_data/dataPrepTrainTestCluster/Baveria/'+city+'/traindata/y_val6000',y_train)
    print( y_train.shape)
    print('done')

In [93]:
train_data('hann_0.003_minpts_2_poi')

whole h5 file


Index(['timestep', 'predicted_accident', 'is_p2', 'is_present', 'geohash',
       'geohash_code', 'year', 'month', 'day', 'twohoursession', 'HOD_cat',
       'DOW_cat', 'DayLight', 'season', 'elevaion', 'zenith', 'azimuth_solar',
       'acc_count', 'middlelat', 'middlelong'],
      dtype='object')

year=1
---------------------
acc > 0


,timestep,predicted_accident,is_p2,is_present,geohash,geohash_code,month,day,twohoursession,year,HOD_cat,DOW_cat,DayLight,season,elevaion,zenith,azimuth_solar
8,117,1,2.0,1.0,0.0,0,3.0,2.0,17.0,2,2,1,1,0,11.712099,78.287901,250.448069
71,171,1,2.0,1.0,3.0,2,2.0,2.0,12.0,2,1,1,1,3,24.446322,65.553678,170.578597
143,288,1,2.0,1.0,5.0,3,3.0,4.0,7.0,2,0,1,1,0,3.279633,86.720367,96.964709
188,90,1,2.0,1.0,10.0,5,5.0,2.0,7.0,2,0,1,1,0,12.248971,77.751029,74.723201
224,522,1,2.0,1.0,11.0,6,4.0,2.0,8.0,2,0,1,1,0,13.355023,76.644977,91.643673
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14700,423,1,2.0,1.0,1952.0,254,5.0,6.0,11.0,2,1,1,1,0,47.205719,42.794281,128.036258
14869,108,1,2.0,1.0,2027.0,265,3.0,1.0,19.0,2,3,0,0,0,-3.895004,93.895004,273.014912
14941,675,1,2.0,1.0,2035.0,266,5.0,3.0,16.0,2,2,1,1,0,44.320735,45.679265,239.256089
14968,702,1,2.0,1.0,2035.0,266,5.0,5.0,7.0,2,0,1,1,0,12.147721,77.852279,74.937144


,timestep,predicted_accident,is_p2,is_present,geohash,geohash_code,month,day,twohoursession,year,HOD_cat,DOW_cat,DayLight,season,elevaion,zenith,azimuth_solar
0,109,0,0.0,0.0,0.0,0,3.0,2.0,9.0,2,0,1,1,0,19.721615,70.278385,122.485822
1,110,0,0.0,0.0,0.0,0,3.0,2.0,10.0,2,0,1,1,0,26.781767,63.218233,136.897191
2,111,0,0.0,0.0,0.0,0,3.0,2.0,11.0,2,1,1,1,0,32.058882,57.941118,153.040695
3,112,0,0.0,0.0,0.0,0,3.0,2.0,12.0,2,1,1,1,0,34.928052,55.071948,170.733810
4,113,0,0.0,0.0,0.0,0,3.0,2.0,13.0,2,1,1,1,0,34.959778,55.040222,189.074319


<class 'pandas.core.frame.DataFrame'>
<class 'numpy.ndarray'>
[[3.]
 [3.]
 [3.]
 ...
 [5.]
 [5.]
 [5.]]
[[0 0 1 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
<class 'pandas.core.frame.DataFrame'>
<class 'numpy.ndarray'>
[[2.]
 [2.]
 [2.]
 ...
 [5.]
 [5.]
 [5.]]
[[0 1 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 ...
 [0 0 0 ... 1 0 0]
 [0 0 0 ... 1 0 0]
 [0 0 0 ... 1 0 0]]


/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:861: UserWarning: unknown class(es) [0] will be ignored
  .format(sorted(unknown, key=str)))


,timestep,predicted_accident,is_p2,is_present,geohash,geohash_code,twohoursession,DOW_cat,DayLight,elevaion,...,2,3,4,0,1,2,3,0,1,2
0,109,0,0.0,0.0,0.0,0,9.0,1,1,19.721615,...,0,0,0,0,0,0,0,0,1,0
1,110,0,0.0,0.0,0.0,0,10.0,1,1,26.781767,...,0,0,0,0,0,0,0,0,1,0
2,111,0,0.0,0.0,0.0,0,11.0,1,1,32.058882,...,0,0,0,0,0,0,0,0,1,0
3,112,0,0.0,0.0,0.0,0,12.0,1,1,34.928052,...,0,0,0,0,0,0,0,0,1,0
4,113,0,0.0,0.0,0.0,0,13.0,1,1,34.959778,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15899,50,0,0.0,0.0,2496.0,359,0.0,1,0,-16.893734,...,0,1,0,0,0,0,0,0,1,0
15900,51,0,0.0,0.0,2496.0,359,1.0,1,0,-18.762915,...,0,1,0,0,0,0,0,0,1,0
15901,52,0,0.0,0.0,2496.0,359,2.0,1,0,-18.267430,...,0,1,0,0,0,0,0,0,1,0
15902,53,0,0.0,0.0,2496.0,359,3.0,1,0,-15.459366,...,0,1,0,0,0,0,0,0,1,0


Index([          'timestep', 'predicted_accident',              'is_p2',
               'is_present',            'geohash',       'geohash_code',
           'twohoursession',            'DOW_cat',           'DayLight',
                 'elevaion',             'zenith',      'azimuth_solar',
                          0,                    1,                    2,
                          3,                    4,                    5,
                          6,                    7,                    8,
                          9,                   10,                   11,
                          0,                    1,                    2,
                          3,                    4,                    5,
                          6,                    0,                    1,
                          2,                    3,                    4,
                          0,                    1,                    2,
                          3,                    0, 

In [12]:

import numpy as np
train1=np.load('/data/dadwal/data/DAP_data/twoyearsepData/traintestdata/osnabrucktraintestdata/baselinewithrain/X_train.npy',allow_pickle=True)
train=np.load('/data/dadwal/data/DAP_data/twoyearsepData/traintestdata/osnabrucktraintestdata/baselinewithrain/y_train.npy',allow_pickle=True)
test=np.load('/data/dadwal/data/DAP_data/twoyearsepData/traintestdata/osnabrucktraintestdata/baselinewithrain/y_train.npy',allow_pickle=True)
print(train1.shape)
train_acc=train[train > 0].size
test_acc=test[test > 0].size
tot_acc=train_acc+test_acc
print('tot_acc in =', tot_acc)

train_nonacc=train[train == 0].size
test_nonacc=test[test == 0].size
tot_noacc=train_nonacc+test_nonacc
print('tot_noacc in =',tot_noacc)
percent=(tot_acc/tot_noacc)*100
print("percent of acc/non-acc in =",percent)

(7798, 200)
tot_acc in = 3202
tot_noacc in = 12394
percent of acc/non-acc in = 25.835081491044054


# gott with rain data as baseline

In [38]:
def train_data(filename):
    #df = pd.read_hdf('/home/dadwal/ls_accident/data/oneKM/onekmfourhour/niedersachsen_onekmfourhour.h5',key='set3')
    
    #df=pd.read_hdf('/home/dadwal/DAP/ls_accident/data/differencitiesaccident/datawithazimuth/h5cities/'+filename+'.h5',key='set3')
    df=pd.read_hdf('/data/dadwal/data/DAP_data/twoyearsepData/h5citiesrain/osntwotwoyearRain.h5',key='set3')
    #df=df.loc[:,'timestep':'UTYP1_7']
    
    acct=df.loc[df['predicted_accident']>0]
    #acct1=df.loc[df['T-Accident']>0]
    
    display(acct)

    #display(acct1)# the .h5 file contains raw traffic, weather, time, and POI data 
    display(df.head())
    df = onhot_enoceder_month(df)
    df=onhot_enoceder(df)
    df=onhot_enoceder_season(df)
    df=df[['timestep', 'predicted_accident', 'geohash', 'geohash_code',
       'twohoursession', 'year', 'rain_val','DOW_cat', 'DayLight','m_en0', 'm_en1',
       'm_en2', 'm_en3', 'm_en4', 'm_en5', 'm_en6', 'm_en7', 'm_en8', 'm_en9',
       'm_en10', 'm_en11', 'HOD_en0', 'HOD_en1', 'HOD_en2', 'HOD_en3',
       'HOD_en4']]
#     df['elevaion'] = scalarval(df['elevaion'])
#     df['zenith'] = scalarval(df['zenith'])
#     df['azimuth_solar'] = scalarval(df['azimuth_solar'])
#     df['acc_count'] = scalarval(df['acc_count'])
    df['rain_val'] = scalarval(df['rain_val'])
    
    print(df.columns)
    #test = onhot_enoceder_month(test)
    df_normalize = df.copy()
    train = df_normalize[df_normalize.timestep <= df_normalize.timestep.max()*5/6]
    test = df_normalize[df_normalize.timestep > df_normalize.timestep.max()*5/6]
    #train.to_csv('train.csv',index=False)
    

#     scaler = MinMaxScaler(feature_range=(0, 1))
#     scaler.fit(train.loc[:,'T-BrokenVehicle':]) 
#     scaled_values = scaler.transform(train.loc[:,'T-BrokenVehicle':]) 
#     train.loc[:,'T-BrokenVehicle':] = scaled_values
#     scaled_values = scaler.transform(test.loc[:,'T-BrokenVehicle':]) 
#     test.loc[:,'T-BrokenVehicle':] = scaled_values
    display(test.head())
    
#     train = onhot_enoceder(train)
#     test = onhot_enoceder(test)
    print(type(train))
    print(type(test))
    
    display(test.head())
    display(test.head().columns)
    
    X_train, y_train = create_sequences(train,geohash_dict)
    X_test, y_test = create_sequences(test,geohash_dict)
    

    # suppose that we have a directory named train_set; in that directory we create several files per city to ...
    # ... represent its train and test data
    np.save('/data/dadwal/data/DAP_data/twoyearsepData/traintestdata/osnabrucktraintestdata/baselinewithrain/X_train',X_train) 
    print (X_train.shape)
    np.save('/data/dadwal/data/DAP_data/twoyearsepData/traintestdata/osnabrucktraintestdata/baselinewithrain/y_train',y_train)
    print( y_train.shape)
    np.save('/data/dadwal/data/DAP_data/twoyearsepData/traintestdata/osnabrucktraintestdata/baselinewithrain/X_test',X_test)
    print (X_test.shape)
    np.save('/data/dadwal/data/DAP_data/twoyearsepData/traintestdata/osnabrucktraintestdata/baselinewithrain/y_test',y_test)
    print (y_test.shape)
    print('done')

In [39]:
train_data('osna')

,timestep,predicted_accident,geohash,geohash_code,twohoursession,year,month,rain_val,HOD_cat,DOW_cat,...,UART_7,UART_8,UART_9,UTYP1_1,UTYP1_2,UTYP1_3,UTYP1_4,UTYP1_5,UTYP1_6,UTYP1_7
91,91,1,u1m9hd,1080,14-16,0,2,0.000,1,0,...,0,0,0,0.0,0.0,1.0,0.0,0,0.0,0.0
1437,429,1,u1m9hd,1080,18-20,0,6,0.000,3,0,...,0,0,0,0.0,0.0,1.0,0.0,0,0.0,0.0
2754,738,1,u1m9hf,1081,12-14,0,9,0.000,1,1,...,0,1,0,1.0,0.0,0.0,0.0,0,0.0,0.0
3608,584,1,u1m9hf,1081,16-18,0,7,0.000,2,0,...,0,0,0,1.0,0.0,0.0,0.0,0,0.0,0.0
5505,465,1,u1m9hg,1082,18-20,0,6,0.000,3,1,...,0,0,0,0.0,1.0,0.0,0.0,0,0.0,0.0
6651,603,1,u1m9hu,1083,6-8,0,8,0.000,0,1,...,0,0,0,0.0,0.0,1.0,0.0,0,0.0,0.0
7302,246,1,u1m9hu,1083,12-14,0,3,0.000,1,0,...,0,0,0,0.0,1.0,0.0,0.0,0,0.0,0.0
7817,761,1,u1m9hu,1083,10-12,0,10,0.000,1,0,...,0,0,0,0.0,1.0,0.0,0.0,0,0.0,0.0
8874,810,1,u1m9hv,1084,12-14,0,10,0.000,1,1,...,0,1,0,1.0,0.0,0.0,0.0,0,0.0,0.0
9437,365,1,u1m9hv,1084,10-12,0,5,0.000,1,1,...,0,0,0,0.0,0.0,0.0,0.0,0,1.0,0.0


,timestep,predicted_accident,geohash,geohash_code,twohoursession,year,month,rain_val,HOD_cat,DOW_cat,...,UART_7,UART_8,UART_9,UTYP1_1,UTYP1_2,UTYP1_3,UTYP1_4,UTYP1_5,UTYP1_6,UTYP1_7
0,0,0,u1m9hd,1080,0-2,0,1,0.0,4,0,...,0,0,0,0.0,0.0,0.0,0.0,0,0.0,0.0
1,1,0,u1m9hd,1080,2-4,0,1,0.0,4,0,...,0,0,0,0.0,0.0,0.0,0.0,0,0.0,0.0
2,2,0,u1m9hd,1080,4-6,0,1,0.0,4,0,...,0,0,0,0.0,0.0,0.0,0.0,0,0.0,0.0
3,3,0,u1m9hd,1080,6-8,0,1,0.0,0,0,...,0,0,0,0.0,0.0,0.0,0.0,0,0.0,0.0
4,4,0,u1m9hd,1080,8-10,0,1,0.0,0,0,...,0,0,0,0.0,0.0,0.0,0.0,0,0.0,0.0


/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a Lab

Index(['timestep', 'predicted_accident', 'geohash', 'geohash_code',
       'twohoursession', 'year', 'rain_val', 'DOW_cat', 'DayLight', 'm_en0',
       'm_en1', 'm_en2', 'm_en3', 'm_en4', 'm_en5', 'm_en6', 'm_en7', 'm_en8',
       'm_en9', 'm_en10', 'm_en11', 'HOD_en0', 'HOD_en1', 'HOD_en2', 'HOD_en3',
       'HOD_en4'],
      dtype='object')


,timestep,predicted_accident,geohash,geohash_code,twohoursession,year,rain_val,DOW_cat,DayLight,m_en0,...,m_en7,m_en8,m_en9,m_en10,m_en11,HOD_en0,HOD_en1,HOD_en2,HOD_en3,HOD_en4
840,840,0,u1m9hd,1080,0-2,0,0.0,0,0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
841,841,0,u1m9hd,1080,2-4,0,0.0,0,0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
842,842,0,u1m9hd,1080,4-6,0,0.0,0,0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
843,843,0,u1m9hd,1080,6-8,0,0.0,0,0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
844,844,0,u1m9hd,1080,8-10,0,0.0,0,1,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0


<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>


,timestep,predicted_accident,geohash,geohash_code,twohoursession,year,rain_val,DOW_cat,DayLight,m_en0,...,m_en7,m_en8,m_en9,m_en10,m_en11,HOD_en0,HOD_en1,HOD_en2,HOD_en3,HOD_en4
840,840,0,u1m9hd,1080,0-2,0,0.0,0,0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
841,841,0,u1m9hd,1080,2-4,0,0.0,0,0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
842,842,0,u1m9hd,1080,4-6,0,0.0,0,0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
843,843,0,u1m9hd,1080,6-8,0,0.0,0,0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
844,844,0,u1m9hd,1080,8-10,0,0.0,0,1,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0


Index(['timestep', 'predicted_accident', 'geohash', 'geohash_code',
       'twohoursession', 'year', 'rain_val', 'DOW_cat', 'DayLight', 'm_en0',
       'm_en1', 'm_en2', 'm_en3', 'm_en4', 'm_en5', 'm_en6', 'm_en7', 'm_en8',
       'm_en9', 'm_en10', 'm_en11', 'HOD_en0', 'HOD_en1', 'HOD_en2', 'HOD_en3',
       'HOD_en4'],
      dtype='object')

------------------------
#########################
process  289  started
process list with length of  6
process  290  started
process list with length of  6
process  291  started
process list with length of  6
process  292  started
process list with length of  6
process  293  started
process list with length of  6
process  294  started
process list with length of  6
process  295  started
process list with length of  6
process  296  started
process list with length of  6
process  297  started
process list with length of  6
process  298  started
process list with length of  6
process  299  started
process list with length of  6
process  300  started
process list with length of  6
process  301  started
process list with length of  6
process  302  started
process list with length of  6
process  303  started
process list with length of  6
process  304  started
process list with length of  6
process  305  started
process list with length of  6
process  306  started
process list with length o

In [40]:

import numpy as np

train=np.load('/data/dadwal/data/DAP_data/twoyearsepData/traintestdata/osnabrucktraintestdata/baselinewithrain/y_train.npy',allow_pickle=True)
test=np.load('/data/dadwal/data/DAP_data/twoyearsepData/traintestdata/osnabrucktraintestdata/baselinewithrain/y_train.npy',allow_pickle=True)

train_acc=train[train > 0].size
test_acc=test[test > 0].size
tot_acc=train_acc+test_acc
print('tot_acc in '+city+'=', tot_acc)

train_nonacc=train[train == 0].size
test_nonacc=test[test == 0].size
tot_noacc=train_nonacc+test_nonacc
print('tot_noacc in '+city+'=',tot_noacc)
percent=(tot_acc/tot_noacc)*100
print("percent of acc/non-acc in "+city+"=",percent)

tot_acc in gott= 3202
tot_noacc in gott= 12394
percent of acc/non-acc in gott= 25.835081491044054
